## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess, source_localization

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [3]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [4]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

['018', '020', '021', '022', '023', '024', '027', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', 'C1.', 'C2.', 'C3.', 'C5.', 'C6.', 'C7.', 'C8.', 'C9.', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C24', 'C25', 'C26', 'C27']


In [5]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

Chronics: 37
Controls: 25
Total: 62


#### Look for subjects who do not have EO or EC cropped data

In [6]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

Subject: 045 missing eyes closed data
Subject: 045 missing eyes open data
Subject: C1. missing eyes closed data
Subject: C1. missing eyes open data
Subject: C5. missing eyes closed data
Subject: C5. missing eyes open data
Subject: C8. missing eyes closed data
Subject: C8. missing eyes open data
Subject: C20 missing eyes closed data
Subject: C20 missing eyes open data
Subject: C21 missing eyes closed data
Subject: C21 missing eyes open data
Subject: C22 missing eyes closed data
Subject: C22 missing eyes open data
['045', 'C1.', 'C5.', 'C8.', 'C20', 'C21', 'C22']
['045', 'C1.', 'C5.', 'C8.', 'C20', 'C21', 'C22']


In [7]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh'] # Right Medial-OFC

### Compute STCs

In [8]:
# Get stc only from selected subjects
sub_ids = ['056']

In [9]:
nan_subjects=[]

methods = [
    'dSPM',
    'MNE',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    EC_bool = False if sub_id in no_eyes_open else True
    EO_bool = False if sub_id in no_eyes_closed else True
    print(f"Subject: {sub_id} | EC: {EC_bool} | EO: {EO_bool}")
    
    for save_stc_mat in [
        # True, 
        False,
        ]:    
        for method in methods:
            # Cannot compute without eyes open if using dSPM
            if not EO_bool and method=='dSPM':
                continue

            # else, compute
            print(f"Source Localization Method: {method}")                          
            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 

            # Create paths
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = EO_bool

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = source_localization.to_source(sub_id,
                                                                              processed_data_path,
                                                                              zscored_epochs_save_path,
                                                                              EC_resting_save_path,
                                                                              EO_resting_save_path,
                                                                              roi_names,
                                                                              times_tup,
                                                                              method=method,
                                                                              return_zepochs=Epochs_bool, # cannot return epochs without eyes open noise segment
                                                                              return_EC_resting = EC_bool,
                                                                              return_EO_resting = EO_bool,
                                                                              average_dipoles=average_dipoles,
                                                                              save_stc_mat=save_stc_mat,
                                                                              save_inv=save_inv,
                                                                              )
            utils.clear_display()

Source Localization Method: MNE
Using up to 600 segments
Number of samples used : 48000
[done]
Computing rank from covariance with rank=None
    Using tolerance 1.4e-12 (2.2e-16 eps * 60 dim * 1.1e+02  max singular value)
    Estimated rank (eeg): 52
    EEG: rank 52 computed from 60 data channels with 1 projector
2 projection items activated
    EEG regularization : 0.1
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank={'eeg': 52}
    Setting small EEG eigenvalues to zero (without PCA)
Z-scoring epochs...


/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/source_localization.py:58: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data_epochs = epochs.get_data()
/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/eeg_toolkit/source_localization.py:60: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  base_data = epochs.get_data(tmin=tmin, tmax=0.0)


TypeError: __init__() got an unexpected keyword argument 'tmax'

### Assess results

In [ ]:
nan_subjects

In [ ]:
label_ts_All_Conds